### Imports & Env Setup

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
import os
from dotenv import load_dotenv
load_dotenv()
from datasets import load_dataset

import dspy
sys.path.append(os.path.abspath('../'))
from benchmarks import llama_mmlu_pro, leaderboard_mmlu_pro

/home/justinai/.conda/envs/prompt-migration/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


### Configuration

In [2]:
NUM_THREADS = 48

FEW_SHOTS = 5

# See https://docs.litellm.ai/docs/providers/vllm for details
TASK_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1' , # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)
PROMPT_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1', # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)

dspy.configure(lm=TASK_MODEL)

# replace this with llama_mmlu_pro or whatever
benchmark = llama_mmlu_pro

# Without chain of thought:
# program = dspy.Predict(
#     benchmark.signature("")
# )

# With chain of thought:
program = dspy.ChainOfThought(
    benchmark.signature("You are a helpful assistant designed to help with multiple choice question.") # put your initial system prompt here, or leave blank
)

evaluate = dspy.Evaluate(
    devset=[],
    max_errors = 500,
    metric=benchmark.metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=True,
    return_all_scores=True,
    return_outputs=True,
)

### Load dataset

In [3]:
trainset, valset, testset = benchmark.datasets(
    train_size=0.1,
    validation_size=0.2,
)

len(trainset), len(valset), len(testset)

(1197, 2156, 8626)

In [4]:
dataset = load_dataset(
    "meta-llama/Llama-3.3-70B-Instruct-evals",
    "Llama-3.3-70B-Instruct-evals__mmlu_pro__details",
)
full_dataset = list(map(benchmark._task_doc_example, dataset["latest"]))

len(full_dataset)

11979

# using promopt from Meta

In [ ]:
program = dspy.ChainOfThought(
    benchmark.signature("Given the following question and candidate answers, choose the best answer.") # put your initial system prompt here, or leave blank
)

In [ ]:
%%time

print("Starting execution...")
# eval_subset_size = len(testset)
score, results, all_scores = evaluate(
    program,
    devset=full_dataset,
)

### Baseline Benchmark

In [5]:
%%time
print("BASE PROMPT:\n", program.predict.signature.instructions)

BASE PROMPT:
 You are a helpful assistant designed to help with multiple choice question.
CPU times: user 0 ns, sys: 82 μs, total: 82 μs
Wall time: 79.2 μs


In [6]:
%%time

print("Starting execution...")
# eval_subset_size = len(testset)
score, results, all_scores = evaluate(
    program,
    devset=full_dataset,
)

Starting execution...


2025/01/22 13:28:20 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Calculate the pH of a 0.10 M solution of sodium acetate, NaOAc. The equilibrium constant for the dissociation of acetic acid is 1.8 × 10^-5.', 'options': {'A': '8.9', 'B': '4.8', 'C': '3.9', 'D': '11.2', 'E': '7.5', 'F': '10.1', 'G': '5.3', 'H': '9.3', 'I': '6.2', 'J': '9.7'}, 'answer': 'A'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.
2025/01/22 13:28:20 ERROR dspy.utils.parallelizer: Error processing item Example({'question': "Use the Runge-Kutta method with $h=0.1$ to find approximate values for the solution of the initial value problem $y' + 2y = x^3e^{-2x}$ with y(0)=1 at $x=0.2$.", 'options': {'A': '0.6423', 'B': '0.5987', 'C': '0.6534', 'D': '0.7012', 'E': '0.7891', 'F': '0.6245', 'G': '0.6705', 'H': '0.7123', 'I': '0.5809', 'J': '0.5607'}, 'answer': ''}) (input_

Average Metric: 4.00 / 6 (66.7%):   0%|                                             | 5/11979 [00:00<10:21, 19.27it/s]

2025/01/22 13:28:56 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A curve with a 120 m radius on a level road is banked at the correct angle for a speed of 20 m/s. If an automobile rounds this curve at 30 m/s, what is the minimum coefficient of static friction needed between tires and road to prevent skidding?', 'options': {'A': '0.60', 'B': '0.28', 'C': '0.75', 'D': '0.25', 'E': '0.45', 'F': '0.55', 'G': '0.65', 'H': '0.15', 'I': '0.34', 'J': '0.42'}, 'answer': 'F'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 4.00 / 6 (66.7%):   0%|                                             | 6/11979 [00:00<11:24, 17.49it/s]

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Average Metric: 8708.00 / 11840 (73.5%): 100%|█████████████████████████████████| 11979/11979 [00:14<00:00, 819.28it/s]


2025/01/22 13:29:10 INFO dspy.evaluate.evaluate: Average Metric: 8708.0 / 11979 (72.7%)


,question,options,example_answer,reasoning,pred_answer,metric,answer
0,Which of the following is not one of the potentially negative con...,"{'A': 'the execution of innocent persons', 'B': 'the diminishing o...",I,To determine which of the following is not one of the potentially ...,I,✔️ [True],NaN
1,Solve the equation 1.2 = 0.4y using mental math.,"{'A': '3', 'B': '−3', 'C': '0.8', 'D': '2', 'E': '0.3', 'F': '5', ...",A,"To solve the equation 1.2 = 0.4y using mental math, we need to iso...",A,✔️ [True],NaN
2,"We must assume that he is a Communist, because he says that he is ...","{'A': 'Argument from Ignorance', 'B': 'Affirming the Consequent', ...",D,The given statement assumes that because a person denies being a C...,A,,NaN
3,When a spinning system contracts in the absence of an external tor...,"{'A': 'halves', 'B': 'doubles', 'C': 'is transferred to linear mom...",D,The question involves understanding the principle of conservation ...,D,✔️ [True],NaN
4,What is the maximum efficiency of a steam engine if the temperatur...,"{'A': '25%', 'B': '18.5%', 'C': '20%', 'D': '15%', 'E': '27.9%', '...",G,"To find the maximum efficiency of a steam engine, we use the Carno...",G,✔️ [True],NaN
...,...,...,...,...,...,...,...
11974,Each of the following is a step taken by a therapist who is employ...,{'A': 'having the client confront the actual feared object or situ...,A,Systematic desensitization is a behavioral technique used to treat...,A,✔️ [True],NaN
11975,"This question refers to the following information. ""But you, my de...",{'A': 'a people without a strong central authority are doomed to l...,J,"The statement ""everything always happens for the best"" reflects th...",J,✔️ [True],NaN
11976,A recent law school graduate took and passed the bar examination. ...,"{'A': 'constitutional, because it safeguards the principle of reha...",A,The state statute regarding the sealing of juvenile records is des...,B,,NaN
11977,LC oscillators are used for produce a waveform with frequency rang...,"{'A': '1 kHz to 100 kHz', 'B': '500 MHz to 1 GHz', 'C': '1 Hz to 1...",E,"LC oscillators, also known as tank circuits or resonant circuits, ...",D,,NaN


CPU times: user 38.6 s, sys: 9.24 s, total: 47.8 s
Wall time: 51.8 s


In [28]:
results.head()

AttributeError: 'list' object has no attribute 'head'

In [ ]:
import csv

with open("my_results.csv", mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # If `results` is a list of lists, just write rows:
    for row in results:
        writer.writerow(row)


In [37]:
import pandas as pd

df = pd.DataFrame([(i, str(pred), score) for i, (example, pred, score) in enumerate(results)],
                     columns=['Example_Index', 'Prediction', 'Score'])
print("\nResults DataFrame:")
print(df)


Results DataFrame:
       Example  \
0            0   
1            1   
2            2   
3            3   
4            4   
...        ...   
11974    11974   
11975    11975   
11976    11976   
11977    11977   
11978    11978   

                                                                  Prediction  \
0      Prediction(\n    reasoning='To determine which of the following is...   
1      Prediction(\n    reasoning='To solve the equation 1.2 = 0.4y using...   
2      Prediction(\n    reasoning='The given statement assumes that becau...   
3      Prediction(\n    reasoning='The question involves understanding th...   
4      Prediction(\n    reasoning='To find the maximum efficiency of a st...   
...                                                                      ...   
11974  Prediction(\n    reasoning='Systematic desensitization is a behavi...   
11975  Prediction(\n    reasoning='The statement "everything always happe...   
11976  Prediction(\n    reasoning="The stat

In [41]:
comparison_df = pd.DataFrame([{
        'Question': example.inputs()['question'][:100] + '...' if len(example.inputs()['question']) > 100 else example.inputs()['question'],
        'Predicted Answer': prediction.answer if hasattr(prediction, 'answer') else str(prediction),
        'Correct Answer': example.answer,
        'Is Correct': '✔️' if score else '❌'
} for example, prediction, score in results])


csv_filename = 'prediction_results.csv'
comparison_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')  # utf-8-sig to handle special characters
print(f"\nResults saved to {csv_filename}")
    
pd.set_option('display.max_colwidth', None)
print("\nPredictions vs Actual Answers:")
print(comparison_df)


Results saved to prediction_results.csv

Predictions vs Actual Answers:
                                                                                                       Question  \
0        Which of the following is not one of the potentially negative consequences of imposing the death pe...   
1                                                              Solve the equation 1.2 = 0.4y using mental math.   
2       We must assume that he is a Communist, because he says that he is not, and we know that Communists a...   
3       When a spinning system contracts in the absence of an external torque its rotational speed increases...   
4       What is the maximum efficiency of a steam engine if the temperature of the input steam is 175° C and...   
...                                                                                                         ...   
11974   Each of the following is a step taken by a therapist who is employing systematic desensitization EXC...   
11975  

In [5]:
%%time

print("Starting execution...")
evaluate(
    program,
    devset=testset,
)

Starting execution...
Average Metric: 202.00 / 260 (77.7%):   3%|█▏                                      | 259/8626 [00:50<29:49,  4.68it/s]

2025/01/22 09:33:20 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Find the minimum of $f(x,y)=2x - 5y$, subject to the constraint $x^2+y^2=144$.', 'options': {'A': '50', 'B': '-64.62', 'C': '144', 'D': '200', 'E': '0', 'F': '-72', 'G': '72', 'H': '-50', 'I': '-200', 'J': '-144'}, 'answer': ''}) (input_keys={'question', 'options'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 294.00 / 387 (76.0%):   4%|█▊                                      | 387/8626 [01:21<34:15,  4.01it/s]

2025/01/22 09:33:50 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'What is $(\\frac{1 + cos(2x) + i*sin(2x)}{1 + cos(2x) - i*sin(2x)})^30$ with $x = \\pi / 60$?', 'options': {'A': '-2.0', 'B': '0.0', 'C': '2i', 'D': 'i', 'E': '-1.0', 'F': '0.5', 'G': '-i', 'H': '2.0', 'I': '1.0', 'J': '-0.5'}, 'answer': ''}) (input_keys={'question', 'options'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 310.00 / 407 (76.2%):   5%|█▉                                      | 409/8626 [01:26<25:20,  5.40it/s]

...

Average Metric: 6285.00 / 8523 (73.7%): 100%|█████████████████████████████████████| 8626/8626 [30:41<00:00,  4.68it/s]

2025/01/22 10:03:11 INFO dspy.evaluate.evaluate: Average Metric: 6285.0 / 8626 (72.9%)


,question,options,example_answer,reasoning,pred_answer,metric,answer
0,A 50-year-old nurse who had been fired from his job at ahospital t...,"{'A': 'No, because sanctions are not proper against the attorney o...",H,The court is likely to grant the hospital's motion for sanctions b...,I,,NaN
1,Child abuse and neglect are most associated with which of the foll...,"{'A': 'secure/engaged', 'B': 'disorganized/disoriented', 'C': 'dis...",B,Child abuse and neglect are most closely associated with a specifi...,B,✔️ [True],NaN
2,One pound of a gas in a piston cylinder arrangement undergoes comp...,"{'A': '15.54 ft^3/lbm', 'B': '345280 ftlbf/lbm', 'C': '260280 ftlb...",I,"To solve this problem, we first need to understand the given condi...",I,✔️ [True],NaN
3,A defendant and his friend were down on their luck. They decided t...,"{'A': 'guilty of felony murder.', 'B': 'not guilty, because the cl...",D,The defendant can be found guilty of his friend's death under the ...,D,✔️ [True],NaN
4,Suppose that we are interested in testing the null hypothesis that...,{'A': 'The test statistic will follow a chi-squared distribution w...,C,"To determine which of the statements are true, let's analyze each ...",C,✔️ [True],NaN
...,...,...,...,...,...,...,...
8621,One's ability to make inferences about the behavior of a populatio...,"{'A': 'reliability', 'B': 'face validity', 'C': 'internal validity...",D,"To answer this question, we need to understand the concept of maki...",D,✔️ [True],NaN
8622,"What is the quotient for the expression 2,314 / 4?","{'A': '508 r1', 'B': '578 r2', 'C': '578 r1', 'D': '509', 'E': '57...",C,"To find the quotient for the expression 2,314 / 4, we need to perf...",B,,NaN
8623,The part of the neuron that directs synthesis of neurotransmitters...,"{'A': 'axon terminal', 'B': 'axon', 'C': 'synaptic cleft', 'D': 'n...",G,The question asks for the part of the neuron responsible for direc...,G,✔️ [True],NaN
8624,"The least common multiple of two integers is 240, and the greatest...","{'A': '72', 'B': '1', 'C': '120', 'D': '20', 'E': '240', 'F': '80'...",C,"To find the other integer, we can use the relationship between the...",C,✔️ [True],NaN


CPU times: user 3min 45s, sys: 7.03 s, total: 3min 52s
Wall time: 30min 42s


(72.86,
 [(Example({'question': 'A 50-year-old nurse who had been fired from his job at ahospital told his attorney, ÒI was fired because of my age, and I want to sue the hospital.Ó Based on this information, the attorney filed an age discrimination complaint against the hospital in federal court. As it turned out, the hospital had hired a 52-year-old man as the nurseÕs replacement, a fact that rendered an age discrimination claim unavailable. The hospital responded tothe complaint by filing a motion for sanctions against thenurseÕs attorney. Is the court likely to grant the hospitalÕs motion?', 'options': {'A': 'No, because sanctions are not proper against the attorney of a represented party.', 'B': 'Yes, because the attorney should have known that age discrimination could not be proven in this case.', 'C': 'No, because the hospital failed to give the attorney the chance to withdraw the complaint in advance of filing the motion with the court.', 'D': "No, because the nurse's complaint

### Optimize Subset + Evaluation

In [9]:
%%time
subset_size = 20
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="light",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/22 12:40:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 20

2025/01/22 12:40:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/22 12:40:40 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/22 12:40:40 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 20%|████████████████▍                                                                 | 4/20 [00:09<00:37,  2.34s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


 20%|████████████████▍                                                                 | 4/20 [00:08<00:33,  2.07s/it]


...


Average Metric: 16.00 / 20 (80.0%): 100%|█████████████████████████████████████████████| 20/20 [00:14<00:00,  1.35it/s]

2025/01/22 12:43:24 INFO dspy.evaluate.evaluate: Average Metric: 16 / 20 (80.0%)
2025/01/22 12:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 80.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/01/22 12:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [70.0, 75.0, 80.0, 80.0, 80.0, 80.0, 80.0]
2025/01/22 12:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/22 12:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/22 12:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 7 / 7 =====



Average Metric: 15.00 / 20 (75.0%): 100%|█████████████████████████████████████████████| 20/20 [00:25<00:00,  1.27s/it]

2025/01/22 12:43:49 INFO dspy.evaluate.evaluate: Average Metric: 15 / 20 (75.0%)
2025/01/22 12:43:49 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
2025/01/22 12:43:49 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [70.0, 75.0, 80.0, 80.0, 80.0, 80.0, 80.0, 75.0]
2025/01/22 12:43:49 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/22 12:43:49 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/22 12:43:49 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 80.0!



CPU times: user 2.49 s, sys: 495 ms, total: 2.98 s
Wall time: 3min 9s


In [10]:
%%time
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 To solve a multiple-choice question, I will carefully read and analyze the question, identify the key concepts and formulas required, and then apply step-by-step reasoning to arrive at the correct answer. I will consider each option and evaluate its validity based on the principles and theories relevant to the subject matter. My response will include a clear explanation of the reasoning process, demonstrating how I arrived at the chosen answer. Please provide the question and options, and I will respond with the reasoning and answer.
CPU times: user 0 ns, sys: 58 μs, total: 58 μs
Wall time: 61.3 μs


In [11]:
%%time
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
)

Average Metric: 293.00 / 378 (77.5%):   4%|█▊                                      | 378/8626 [01:28<33:12,  4.14it/s]

2025/01/22 12:46:10 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Consider the initial value problem\n$$\n3 u^{\\prime \\prime}-u^{\\prime}+2 u=0, \\quad u(0)=2, \\quad u^{\\prime}(0)=0\n$$\nFor $t>0$ find the first time at which $|u(t)|=10$.', 'options': {'A': '14.1234', 'B': '9.5678', 'C': ' 10.7598', 'D': '16.4321', 'E': '15.6789', 'F': '11.2345', 'G': '12.3456', 'H': '7.6543', 'I': '13.5791', 'J': '8.9876'}, 'answer': ''}) (input_keys={'question', 'options'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 1756.00 / 2320 (75.7%):  27%|█████████▉                           | 2321/8626 [09:22<27:03,  3.88it/s]

2025/01/22 12:54:04 ERROR dspy.utils.parallelizer: Error processing item Example({'question': "What popular toy is featured in the film 'The Hudsucker Proxy'?", 'options': {'A': 'Lincoln Logs', 'B': 'Beanie Babies', 'C': 'Hot Wheels', 'D': 'Teddy Ruxpin', 'E': 'Slinky', 'F': "Rubik's Cube", 'G': 'Barbie Doll', 'H': 'Yo-Yo', 'I': 'Hula Hoop', 'J': 'Lite Brite'}, 'answer': 'I'}) (input_keys={'question', 'options'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 1763.00 / 2330 (75.7%):  27%|█████████▉                           | 2331/8626 [09:23<17:32,  5.98it/s]

...

2025/01/22 13:20:13 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'In a heavily polluted industrialized area in England about 87 percent of the moths of the speciesBistonbetulariaare melanic(dark colored). If the frequency of recessives is 0.13, what is the frequency of the dominant allele leading to melanism? What proportion of the dark colored moths are heterozygous?', 'options': {'A': '0.80, 0.48 or 48%', 'B': '0.55, 0.75 or 75%', 'C': '0.50, 0.50 or 50%', 'D': '0.60, 0.70 or 70%', 'E': '0.64, 0.53 or 53%', 'F': '0.77, 0.30 or 30%', 'G': '0.70, 0.40 or 40%', 'H': '0.87, 0.25 or 25%', 'I': '0.75, 0.60 or 60%', 'J': '0.47, 0.65 or 65%'}, 'answer': 'E'}) (input_keys={'question', 'options'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 6451.00 / 8591 (75.1%): 100%|█████████████████████████████████████| 8626/8626 [35:32<00:00,  4.05it/s]

2025/01/22 13:20:13 INFO dspy.evaluate.evaluate: Average Metric: 6451.0 / 8626 (74.8%)


,question,options,example_answer,reasoning,pred_answer,metric,answer
0,7.3-9. Consider the following two groups of women: Group 1 consist...,"{'A': ' $0.2115$', 'B': '$0.3256$', 'C': '$0.1298$', 'D': '$0.2732...",A,"To find a point estimate of $p_1 - p_2$, we first need to calculat...",A,✔️ [True],NaN
1,Find the product of the given polynomials in the given polynomial ...,"{'A': 'x^2 - 1', 'B': '2x^3 - 4x^2 + 2x', 'C': '8x^3 - 20x^2 + 10x...",F,To find the product of the given polynomials f(x) = 4x - 5 and g(x...,F,✔️ [True],NaN
2,Classical and neoclassical (marginalist) theorists have usually su...,{'A': 'Keynes supported a market economy with equal government and...,J,"The question requires an understanding of Keynes' economic views, ...",J,✔️ [True],NaN
3,Calculate the melting point of ice under a pressure of 50 bar. Ass...,"{'A': '277.3 K', 'B': '276.8 K', 'C': ' 272.8 $\\mathrm{K}$', 'D':...",,"To calculate the melting point of ice under a pressure of 50 bar, ...",B,,NaN
4,Which of the following are the three broad groups of organization...,"{'A': 'Organizational size, industry type, and geographical locati...",A,The question asks for the three broad groups of organizational cha...,A,✔️ [True],NaN
...,...,...,...,...,...,...,...
8621,What role does language play in the process of securitization?,{'A': 'The process of securitization is primarily a non-discursive...,B,The correct answer can be determined by understanding the role of ...,B,✔️ [True],NaN
8622,What is the effective rates for 3% compounded monthly?,"{'A': '0.027', 'B': '0.0304', 'C': '0.035', 'D': '0.036', 'E': '0....",B,"To find the effective rate for 3% compounded monthly, we use the f...",B,✔️ [True],NaN
8623,Methane is stored at low temperature inside a tank having a volume...,"{'A': '89,368.79 kJ', 'B': '95,500 kJ', 'C': '120,000 kJ', 'D': '1...",A,"To solve this problem, we need to consider the thermodynamic prope...",A,✔️ [True],NaN
8624,The bandwidth of an analog signal is 4kHz. An A/D converter is use...,"{'A': '10', 'B': '7', 'C': '4', 'D': '6', 'E': '8', 'F': '12', 'G'...",E,"To eliminate the aliasing problem, the sampling rate of the A/D co...",E,✔️ [True],NaN


CPU times: user 2min 20s, sys: 6.65 s, total: 2min 27s
Wall time: 35min 33s


## Medium Optimization

In [42]:
%%time
subset_size = 500
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="medium",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/22 14:00:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 300

2025/01/22 14:00:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/22 14:00:40 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/22 14:00:40 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  1%|▊                                                                               | 5/500 [00:00<00:01, 461.89it/s]


Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/19


  1%|▋                                                                               | 4/500 [00:00<00:00, 500.68it/s]


...


Average Metric: 224.00 / 297 (75.4%):  99%|████████████████████████████████████████▌| 297/300 [01:24<00:03,  1.16s/it]

2025/01/22 14:15:05 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A bricklayer is supplied with bricks by his mate who is 10 ft below him, the mate tossing the bricks vertically upward. If the bricks have a speed of 6 ft/s when they reach the bricklayer, what percentage of the energy used up by the mate serves no useful purpose?', 'options': {'A': '10.6%', 'B': '5.3%', 'C': '2.5%', 'D': '12.4%', 'E': '18.5%', 'F': '8.1%', 'G': '21.2%', 'H': '15.0%', 'I': '7.9%', 'J': '3.7%'}, 'answer': 'A'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 224.00 / 299 (74.9%): 100%|█████████████████████████████████████████| 300/300 [01:40<00:00,  2.98it/s]

2025/01/22 14:15:14 INFO dspy.evaluate.evaluate: Average Metric: 224.0 / 300 (74.7%)
2025/01/22 14:15:14 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.33, 75.33, 75.0, 74.67]
2025/01/22 14:15:14 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 75.33
2025/01/22 14:15:14 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/22 14:15:14 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/22 14:15:14 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 75.33!



CPU times: user 24.7 s, sys: 3.9 s, total: 28.6 s
Wall time: 14min 34s


In [43]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 You are a highly skilled expert in a high-stakes testing environment, and it's crucial that you provide accurate and step-by-step solutions to complex multiple-choice questions. Your task is to read each question carefully, analyze the given options, and then use your knowledge and reasoning skills to select the correct answer. For each question, please provide a detailed reasoning process, including any relevant formulas, calculations, or logical deductions that lead you to the correct answer. The format of your response should include the question, followed by a step-by-step reasoning process, and finally, the correct answer. Remember, the accuracy and clarity of your responses are critical in this high-pressure scenario, and your ability to think critically and solve problems efficiently will be thoroughly assessed.


In [44]:
print("BEST EXAMPLES:\n", optimized_program.predict.demos)

BEST EXAMPLES:
 [Example({'question': 'The defendant, who was walking on a public street, got into a heated argument with a stranger who had been singing and shouting obscenities while walking near him. The defendant pulled a large serrated hunting knife and stabbed the victim in the chest, killing him. The authorities charged defendant with murder. The defendant argued at trial that he only intended to scare the victim to shut him up, and that he did not have the intent to kill. Is the defendant guilty of murder?', 'options': {'A': 'Yes, his use of a deadly weapon demonstrated the requisite intent to kill.', 'B': 'No, his behavior was reckless but not intentional.', 'C': 'No, the defendant was just trying to scare the victim, not kill him.', 'D': "No, defendant was trying to protect himself from the victim's aggressive behavior.", 'E': "No, defendant was in a heated argument and his judgement was clouded, so he didn't have the intent to kill.", 'F': "No, defendant responded to the vic

In [45]:
%%time
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
)

Average Metric: 222.00 / 291 (76.3%):   3%|█▎                                      | 290/8626 [01:29<45:52,  3.03it/s]

2025/01/22 14:18:38 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Company A has found that if it offers a guarantee on its transistor radios, more will be sold than without the guarantee. In a year, the company expects to sell 100 radios if no guarantee is offered, but if the length of the guarantee is t years then the number sold will be Q = 100 + 5t. It is known that 10% of radios sold will fail each year. The cost of fixing a radio is $1 and a profit of $2.80 is made on every radio sold. What is the length of the guarantee period t that maximizes profit?', 'options': {'A': '2 years', 'B': '3 years', 'C': '6 years', 'D': '4 years', 'E': '8 years', 'F': '10 years', 'G': '9 years', 'H': '7 years', 'I': '1 year', 'J': '5 years'}, 'answer': 'D'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 401.00 / 534 (75.1%):   6%|██▍                                     | 534/8626 [02:46<46:59,  2.87it/s]

2025/01/22 14:19:56 ERROR dspy.utils.parallelizer: Error processing item Example({'question': '"Top gun" pilots have long worried about taking a turn too tightly. As a pilot\'s body undergoes centripetal acceleration, with the head toward the center of curvature, the blood pressure in the brain decreases, leading to loss of brain function.\nThere are several warning signs. When the centripetal acceleration is $2 g$ or $3 g$, the pilot feels heavy. At about $4 g$, the pilot\'s vision switches to black and white and narrows to "tunnel vision." If that acceleration is sustained or increased, vision ceases and, soon after, the pilot is unconscious - a condition known as $g$-LOC for " $g$-induced loss of consciousness."\n\nWhat is the magnitude of the acceleration, in $g$ units, of a pilot whose aircraft enters a horizontal circular turn with a velocity of $\\vec{v}_i=(400 \\hat{\\mathrm{i}}+500 \\hat{\\mathrm{j}}) \\mathrm{m} / \\mathrm{s}$ and $24.0 \\mathrm{~s}$ later leaves the turn wit

Average Metric: 457.00 / 609 (75.0%):   7%|██▊                                     | 611/8626 [03:09<47:12,  2.83it/s]

...

2025/01/22 15:00:23 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Calculate the heat of formation of CH_4(g) in the reaction: C(graphite) + 2H_2(g) = CH_4(g) , using the reactions below and their given \\DeltaH° values CH_4(g) + 2O_2(g) = CO_2(g) + 2H_2O(1) \\DeltaH° = -212.80 kcal mole^-1 C(graphite) + C_2 (g) = CO_2 (g)\\DeltaH° = -94.05 kcal mole^-1 H_2(g) + (1/2) O_2(g) = H_2O(1)\\DeltaH° = -68.32 kcal mole^-1', 'options': {'A': '- 50.26 kcal mole^-1', 'B': '- 17.89 kcal mole^-1', 'C': '- 76.41 kcal mole^-1', 'D': '- 68.32 kcal mole^-1', 'E': '- 158.17 kcal mole^-1', 'F': '- 212.80 kcal mole^-1', 'G': '- 94.05 kcal mole^-1', 'H': '- 205.47 kcal mole^-1', 'I': '- 123.93 kcal mole^-1', 'J': '- 180.37 kcal mole^-1'}, 'answer': 'B'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 6447.00 / 8588 (75.1%): 100%|█████████████████████████████████████| 8626/8626 [43:36<00:00,  3.30it/s]

2025/01/22 15:00:46 INFO dspy.evaluate.evaluate: Average Metric: 6447.0 / 8626 (74.7%)


,question,options,example_answer,reasoning,pred_answer,metric,answer
0,Determine the mass of water to which 293 g ofNaCl (formula weight ...,"{'A': '10 kg H_2O', 'B': '35 kg H_2O', 'C': '20 kg H_2O', 'D': '25...",C,"To determine the mass of water needed, we first need to calculate ...",C,✔️ [True],NaN
1,Research on Helms’s White Racial Identity Development Model sugges...,"{'A': 'contact', 'B': 'immersion-emersion', 'C': 'pseudo-independe...",D,Helms's White Racial Identity Development Model outlines several s...,D,✔️ [True],NaN
2,A state has recently enacted a statute making it a misdemeanor for...,"{'A': 'deny the motion, because the judge granted a mistrial.', 'B...",F,The key issue here is whether the prosecution of the defendant by ...,B,,NaN
3,How many positive integers are factors of 120 and also factors of 40?,"{'A': '2', 'B': '1', 'C': '4', 'D': '8', 'E': '5', 'F': '7', 'G': ...",D,To find the number of positive integers that are factors of both 1...,D,✔️ [True],NaN
4,"After graduating from law school, a graduate moved to a city where...","{'A': 'Yes, because the graduate did not intend to defraud the com...",D,The graduate's claim against the bar-review company is based on in...,I,,NaN
...,...,...,...,...,...,...,...
8621,What is the main difference between the short-run and long-run Phi...,{'A': 'The short-run Phillips curve is downward sloping and the lo...,B,The main difference between the short-run and long-run Phillips cu...,B,✔️ [True],NaN
8622,What is the greatest possible area of a triangular region with one...,"{'A': 'sqrt(pi)', 'B': 'pi', 'C': '1/2', 'D': '2', 'E': 'sqrt(2)',...",C,To find the greatest possible area of a triangular region with one...,I,,NaN
8623,"In Python 3, which of the following function convert a string to a...","{'A': 'short(x)', 'B': 'float(x)', 'C': 'integer(x [,base])', 'D':...",E,The function that converts a string to an integer in Python 3 is i...,E,✔️ [True],NaN
8624,"Let $P(r,t,T)$ denote the price at time $t$ of $1 to be paid with ...","{'A': '0.11', 'B': '0.06', 'C': '0.05', 'D': '0.04', 'E': '0.08', ...",B,The Vasicek model is a short-rate model that describes the evoluti...,B,✔️ [True],NaN


CPU times: user 2min 26s, sys: 7.7 s, total: 2min 34s
Wall time: 43min 37s


## Heavy Optimization

In [47]:
%%time
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="heavy",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset,
    valset=valset,
)

2025/01/22 15:06:42 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING HEAVY AUTO RUN SETTINGS:
num_trials: 50
minibatch: True
num_candidates: 38
valset size: 1000



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 38 * 1 lm calls in program + (2) lm calls in program-aware proposer = 50 prompt model calls
- Program Evaluation: 25 examples in minibatch * 50 batches + 1000 examples in val set * 6 full evals = 7250 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expec

2025/01/22 15:06:45 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/22 15:06:45 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/22 15:06:45 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=38 sets of demonstrations...


Bootstrapping set 1/38
Bootstrapping set 2/38
Bootstrapping set 3/38


  0%|▎                                                                               | 5/1197 [00:10<42:48,  2.15s/it]


...


Average Metric: 620.00 / 806 (76.9%):  81%|████████████████████████████████▏       | 806/1000 [04:23<01:14,  2.62it/s]

2025/01/22 16:06:32 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'The mass density of water vapour at $327.6 \\mathrm{~atm}$ and $776.4 \\mathrm{~K}$ is $133.2 \\mathrm{~kg} \\mathrm{~m}^{-3}$. Given that for water $T_{\\mathrm{c}}=647.4 \\mathrm{~K}, p_{\\mathrm{c}}=218.3 \\mathrm{~atm}, a=5.464 \\mathrm{dm}^6 \\mathrm{~atm} \\mathrm{~mol}^{-2}$, $b=0.03049 \\mathrm{dm}^3 \\mathrm{~mol}^{-1}$, and $M=18.02 \\mathrm{~g} \\mathrm{~mol}^{-1}$, calculate the compression factor from the virial expansion of the van der Waals equation.', 'options': {'A': '0.9283', 'B': '0.6889', 'C': '0.5594', 'D': '0.8457', 'E': '0.7671', 'F': '0.7846', 'G': '0.6523', 'H': '0.8312', 'I': ' 0.7158', 'J': '0.6017'}, 'answer': ''}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 752.00 / 999 (75.3%): 100%|███████████████████████████████████████| 1000/1000 [05:46<00:00,  2.88it/s]

2025/01/22 16:07:55 INFO dspy.evaluate.evaluate: Average Metric: 752.0 / 1000 (75.2%)
2025/01/22 16:07:55 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [0.0, 0.0, 75.8, 74.0, 75.1, 75.2]
2025/01/22 16:07:55 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 75.8
2025/01/22 16:07:55 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/22 16:07:55 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/22 16:07:55 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 75.8!



CPU times: user 1min 33s, sys: 9.69 s, total: 1min 42s
Wall time: 1h 1min 12s


In [48]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 To address the multiple-choice questions effectively, I will provide a detailed, step-by-step reasoning process for each question. Given a question and its corresponding options, I will analyze the content, identify key concepts, and apply relevant principles to deduce the correct answer. My approach involves understanding the question's context, evaluating each option against the question's requirements, and selecting the answer that best aligns with the principles and concepts related to the topic at hand. This method ensures a systematic and informed decision-making process, leading to the identification of the correct answer among the provided options.


In [49]:
print("BEST EXAMPLES:\n", optimized_program.predict.demos)

BEST EXAMPLES:
 [Example({'augmented': True, 'question': 'What is collusion?', 'options': {'A': 'A business strategy to maximize profits', 'B': 'A financial arrangement where investors pool their funds for a joint venture', 'C': 'An economic policy that encourages competition among businesses', 'D': 'A marketing technique', 'E': 'A partnership where two companies legally share resources and profits', 'F': 'An informal agreement between employees to improve workplace conditions', 'G': 'A legal framework for merging two or more companies into a single corporate entity', 'H': 'A legal agreement between companies', 'I': 'The practice whereby several firms get together and secretly agree upon how their market should be divided or what prices should be charged.', 'J': 'A government policy to regulate monopolies'}, 'reasoning': 'Collusion refers to a secret agreement between two or more parties to deceive or cheat others, often by dividing a market or fixing prices. This practice is generally

In [50]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
    display_table=False,
)

Average Metric: 445.00 / 574 (77.5%):   7%|██▋                                     | 574/8626 [02:32<35:45,  3.75it/s]

2025/01/22 16:29:01 ERROR dspy.utils.parallelizer: Error processing item Example({'question': '"Top gun" pilots have long worried about taking a turn too tightly. As a pilot\'s body undergoes centripetal acceleration, with the head toward the center of curvature, the blood pressure in the brain decreases, leading to loss of brain function.\nThere are several warning signs. When the centripetal acceleration is $2 g$ or $3 g$, the pilot feels heavy. At about $4 g$, the pilot\'s vision switches to black and white and narrows to "tunnel vision." If that acceleration is sustained or increased, vision ceases and, soon after, the pilot is unconscious - a condition known as $g$-LOC for " $g$-induced loss of consciousness."\n\nWhat is the magnitude of the acceleration, in $g$ units, of a pilot whose aircraft enters a horizontal circular turn with a velocity of $\\vec{v}_i=(400 \\hat{\\mathrm{i}}+500 \\hat{\\mathrm{j}}) \\mathrm{m} / \\mathrm{s}$ and $24.0 \\mathrm{~s}$ later leaves the turn wit

Average Metric: 503.00 / 649 (77.5%):   8%|███                                     | 650/8626 [02:51<29:44,  4.47it/s]

2025/01/22 16:29:20 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A microcomputer used for data acquisition and control is required to digitize and process four analog input signals and to output their average continually; i.e., in real time. The time for an external analog-to-digital converter (which is triggered by a CPU instruction) to digitize one input is 12 microseconds, and only one digitization occurs at a time. Five CPU instructions, including the triggering instruction, must be executed for each signal digitized. Ten CPU instructions are executed in order to average each set of four samples and output this value. The time to convert the output from digital to analog form is to be ignored. If it is assumed that suitable data buffering is employed, then the maximum average instruction execution time that allows the microcomputer to keep up with the input-output data rates, is', 'options': {'A': '3.0 microseconds', 'B': '2.6 microseconds', 'C': '2.4 

Average Metric: 600.00 / 769 (78.0%):   9%|███▌                                    | 771/8626 [03:25<35:15,  3.71it/s]

...

2025/01/22 17:04:01 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'The collisional cross section of $\\mathrm{N}_2$ is $0.43 \\mathrm{~nm}^2$. What is the diffusion coefficient of $\\mathrm{N}_2$ at a pressure of $1 \\mathrm{~atm}$ and a temperature of $298 \\mathrm{~K}$ ?', 'options': {'A': ' 1.06 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'B': '2.00 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'C': '0.90 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'D': '1.80 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'E': '1.33 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'F': '0.60 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'G': '1.20 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'H': '1.11 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'I': '1.50 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$', 'J': '0.75 $10^{-5} \\mathrm{~m}^2 \\mathrm{~s}^{-1}$'}, 'answer': ''}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reaso

Average Metric: 6573.00 / 8582 (76.6%): 100%|████████████████████████████████████▉| 8618/8626 [38:06<00:07,  1.12it/s]

2025/01/22 17:04:36 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'For HF,K_diss= 6.7 × 10^-4 what is the H_3O^+ and OH^- concentrations of a 0.10M HF solution? Assume K_W = 1 × 10^-14 .', 'options': {'A': '[H_3O^+] = 6.7 × 10^-3M, [OH^-] = 1.0 × 10^-12M', 'B': '[H_3O^+] = 6.7 × 10^-4M, [OH^-] = 1.0 × 10^-11M', 'C': '[H_3O^+] = 5.1 × 10^-5M, [OH^-] = 2.0 × 10^-10M', 'D': '[H_3O^+] = 1.0 × 10^-4M, [OH^-] = 1.0 × 10^-10M', 'E': '[H_3O^+] = 2.6 × 10^-4M, [OH^-] = 3.8 × 10^-11M', 'F': '[H_3O^+] = 9.0 × 10^-3M, [OH^-] = 1.1 × 10^-12M', 'G': '[H_3O^+] = 8.2 × 10^-4M, [OH^-] = 1.2 × 10^-11M', 'H': '[H_3O^+] = 8.1 × 10^-5M, [OH^-] = 1.2 × 10^-10M', 'I': '[H_3O^+] = 7.7 × 10^-4M, [OH^-] = 1.3 × 10^-11M', 'J': '[H_3O^+] = 7.7 × 10^-3M, [OH^-] = 1.3 × 10^-12M'}, 'answer': 'E'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 6578.00 / 8589 (76.6%): 100%|█████████████████████████████████████| 8626/8626 [38:29<00:00,  3.73it/s]

2025/01/22 17:04:58 INFO dspy.evaluate.evaluate: Average Metric: 6578.0 / 8626 (76.3%)
